In [1]:
import os
import json
import pandas as pd
import numpy as np

from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from sklearn.utils import resample


## Read Data
You can change the code below to read your own data.

In [2]:
data_name = 'income'

df = pd.read_csv(filepath_or_buffer="./input/adult.data", header=None, index_col=None,sep=',')
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
col_names = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education-num",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital-gain",
    "capital-loss",
    "hours-per-week",
    "native-country",
    'income',
]

In [4]:
df.columns = col_names
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [5]:
df.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64

In [6]:
df.columns.isna()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

In [7]:
df.isin(['?']).sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64

In [8]:
df['income']

0         <=50K
1         <=50K
2         <=50K
3         <=50K
4         <=50K
          ...  
32556     <=50K
32557      >50K
32558     <=50K
32559     <=50K
32560      >50K
Name: income, Length: 32561, dtype: object

In [9]:
df['income'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [10]:
df['income'] = df['income'].replace({
    ' <=50K': 0,
    ' >50K': 1
})

target_names = ['<=50K', '>50K']

In [11]:
np.unique(df['income'], return_counts=True)

(array([0, 1]), array([24720,  7841]))

In [12]:
# Separate majority and minority classes
df_majority = df[df.income==0]
df_minority = df[df.income==1]

# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=df_minority.shape[0],     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.income.value_counts()

1    7841
0    7841
Name: income, dtype: int64

In [13]:
df['income'].head()

0    0
1    0
2    0
3    0
4    0
Name: income, dtype: int64

In [14]:
numerical_df = df_downsampled.select_dtypes(exclude=['object'])
numerical_df.columns

Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 'income'],
      dtype='object')

## Train a classification model
In this step, you can either train a model in the cell below, or skip the training part and read the model prediction of the training data from your own model .

In [15]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler
from sklearn.svm import SVC

In [16]:
'''prepare data'''
target_col = 'income'
X = numerical_df.drop(columns=[target_col]).values
y = numerical_df[target_col].values
y = y.reshape(len(y))

train, test, train_labels, test_labels = train_test_split(X, y, test_size = 0.2)

train_df = pd.DataFrame(train, columns=numerical_df.columns[:-1].values)
test_df = pd.DataFrame(test, columns=numerical_df.columns[:-1].values)

In [17]:
'''train the model'''
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(train, train_labels)

'''report accuracy'''
print(clf.score(test,test_labels ))

0.7647433854000637


In [18]:
''' obtain the original model's prediction '''
y_pred = clf.predict(train)

## Output Training Data

In [19]:
'''
cols: the column names of the input data.
data: the training input.
target_names: the names for the target classes (grount truth).
real_min: minimal values for columns of the input data.
real_max: maximal values for columns of the input data.
y_pred: the prediction of the input data from the model you want to explain.
y_gt: the ground truth of the input data.
'''

def output_data(cols, data, target_names, real_min, real_max, y_pred, y_gt):
    filename = "./output/"+data_name+"/test.json"
    directory = os.path.dirname(filename)
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    to_output = {}
    to_output['columns'] = cols
    to_output['data'] = data
    to_output['target_names'] = target_names
    to_output['real_min'] = real_min
    to_output['real_max'] = real_max
    to_output['y_pred'] = y_pred
    to_output['y_gt'] = y_gt
    with open(filename, 'w') as output:
        output.write(json.dumps(to_output))

In [20]:
''' name the target classes '''
# target_names = ["False", "True"]

''' obtain the min and max values '''
min_val = np.min(X, axis=0)
max_val = np.max(X, axis=0)

''' output data '''
output_data(cols=numerical_df.drop(columns=[target_col]).columns.values.tolist(), 
            data=train.tolist(), 
            target_names=target_names, 
            real_min=min_val.tolist(), 
            real_max=max_val.tolist(),
            y_pred=y_pred.tolist(), 
            y_gt=train_labels.tolist())